In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [21]:
businesses = pd.read_csv('Business_Licenses.csv')

In [22]:
businesses.head()

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION
0,22308-20060816,1723393,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,...,08/16/2006,08/15/2007,08/10/2006,08/11/2006,AAI,NaN,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)"
1,1620668-20160516,2455262,295026,1,BUCCI BIG & TALL INC.,BUCCI BIG & TALL INC.,558 W ROOSEVELT RD,CHICAGO,IL,60607,...,05/16/2016,05/15/2018,05/18/2016,08/30/2016,AAI,NaN,NaN,41.867339,-87.641590,"(41.86733856638269, -87.64159005699716)"
2,2368602-20160616,2460909,291461,3,"PROJECT: VISION , INC.","PROJECT : VISION , INC",2301 S ARCHER AVE 1 1,CHICAGO,IL,60616,...,06/16/2016,06/15/2017,06/21/2016,06/22/2016,AAC,08/30/2016,NaN,41.850843,-87.638734,"(41.85084294374687, -87.63873424399071)"
3,2060891-20141016,2353257,357247,1,FOLASHADE'S CLEANING SERVICE INC.,FOLASHADE'S CLEANING SERVICE INC.,1965 BERNICE RD 1 1SW,LANSING,IL,60438,...,10/16/2014,10/15/2016,04/01/2016,04/01/2016,AAI,NaN,38.0,41.951316,-87.678586,"(41.95131555606832, -87.67858578019546)"
4,1144216-20070516,1804790,147,63,WALGREEN CO.,Walgreens # 05192,9148 S COMMERCIAL AVE 1ST,CHICAGO,IL,60617,...,05/16/2007,05/15/2008,05/10/2007,05/11/2007,AAI,NaN,5.0,41.728622,-87.551366,"(41.72862173556932, -87.55136646594693)"


In [38]:
# clean latitude and longitude columns

#dropnans 
businesses = businesses[np.isfinite(businesses['LONGITUDE'])]
businesses = businesses[np.isfinite(businesses['LATITUDE'])]

In [43]:
#Copy over jkgiesler's code to attach neighborhoods based on longitude and latitude

import json
import os
from collections import namedtuple
import sys
from math import log,tan,pi,radians

#globals
Pt = namedtuple('Pt','x,y')
Edge = namedtuple('Edge','a,b')
Poly = namedtuple('Poly','name,edges')
_eps = 1e-5
_huge = sys.float_info.max
_tiny = sys.float_info.min

def load_json():
    file_in = open("Neighborhoods_2012_polygons.json")
    d = json.load(file_in)
    return d


def spherical_mercator_projection(longitude,latitude):
    #http://en.wikipedia.org/wiki/Mercator_projection <- invented in 1569!
    #http://stackoverflow.com/questions/4287780/detecting-whether-a-gps-coordinate-falls-within-a-polygon-on-a-map
    x = -longitude
    y = log(tan(radians(pi/4 + latitude/2)))
    return (x,y)


def rayintersectseg(p, edge):
    #http://rosettacode.org/wiki/Ray-casting_algorithm#Python
    #takes a point p=Pt() and an edge of two endpoints a,b=Pt() of a line segment returns boolean
    a,b = edge
    if a.y > b.y:
        a,b = b,a
    if p.y == a.y or p.y == b.y:
        p = Pt(p.x, p.y + _eps)
    intersect = False
 
    if (p.y > b.y or p.y < a.y) or (
        p.x > max(a.x, b.x)):
        return False
 
    if p.x < min(a.x, b.x):
        intersect = True
    else:
        if abs(a.x - b.x) > _tiny:
            m_red = (b.y - a.y) / float(b.x - a.x)
        else:
            m_red = _huge
        if abs(a.x - p.x) > _tiny:
            m_blue = (p.y - a.y) / float(p.x - a.x)
        else:
            m_blue = _huge
        
        intersect = m_blue >= m_red
    return intersect
 
def is_odd(x): 
    return x%2 == 1

def ispointinside(p, poly):
    ln = len(poly)
    return is_odd(sum(rayintersectseg(p, edge)
                    for edge in poly.edges ))

def get_all_neighborhoods():
    d = load_json()
    shape_list=[]
    for shape_idx in range(len(d['features'])):
        name = d['features'][shape_idx]['properties']['SEC_NEIGH']

        edges =[]
        for coordinate_idx in range(len(d['features'][shape_idx]['geometry']['coordinates'][0])-1):
            lon_1 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx][0]
            lat_1 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx][1]
            
            lon_2 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx+1][0]
            lat_2 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx+1][1]
            
            x1,y1 = spherical_mercator_projection(lon_1,lat_1)
            x2,y2 = spherical_mercator_projection(lon_2,lat_2)
            edges.append(Edge(a=Pt(x=x1,y=y1),b=Pt(x=x2,y=y2)))
        
        shape_list.append(Poly(name=name,edges=tuple(edges)))
    return shape_list

all_neighborhoods = get_all_neighborhoods()

def find_neighborhood(test_long,test_lat,all_neighborhoods):
    x,y = spherical_mercator_projection(test_long,test_lat)
    for neighborhood in all_neighborhoods:
        correct_neighborhood = ispointinside(Pt(x=x,y=y),neighborhood)
        if correct_neighborhood:
            return neighborhood.name

In [48]:
#convert longitude and latitude columns to float

long = businesses['LONGITUDE'].astype(float)
lat = businesses['LATITUDE'].astype(float)

In [63]:
businesses['neighborhood'] = find_neighborhood(long,lat,all_neighborhoods)

In [57]:
#make longitude and latitude into lists

lonlist = businesses['LONGITUDE'].tolist()
latlist = businesses['LATITUDE'].tolist()
#dfList = list(df['one']

In [62]:
neighbor = []
neighbor = find_neighborhood(lonlist,latlist,all_neighborhoods)